### specify the spark directory and path

In [1]:
import os,sys
os.environ["SPARK_HOME"] = '/usr/hdp/current/spark-client'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-csv_2.11:1.2.0 pyspark-shell'
sys.path.insert(0, os.path.join(os.environ["SPARK_HOME"], 'python'))

### import required packages and spark

In [177]:
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark import sql
from pyspark.sql import SQLContext
from pyspark.sql.functions import avg
from pyspark.sql.functions import col

In [3]:
import string
import pandas as pd

### use this to stop the spark context before exiting

In [55]:
#sc.stop()

### sc is already initiated. Uncomment below and run to initiate it again

In [4]:
#sc is already initiated. Uncomment below and run to initiate it again
conf = SparkConf()
conf.set('spark.ui.port',60133)
conf.setAppName("StopYourCodes")
conf.set('spark.local.dir',"/home/pkalavai2")
conf.set('spark.shuffle.file.buffer','64k')
conf.set('spark.driver.extraJavaOptions', "-Dspark.local.dir=/mnt/spark,/mnt2/spark -Dhadoop.tmp.dir=/mnt/ephemeral-hdfs")
sc = SparkContext(conf=conf)

In [22]:
### sc is already initiated. Uncomment below and run to initiate it again
#conf = SparkConf()
#conf.set('spark.ui.port',60133)
#conf.set('spark.local.dir', "/home/pkalavai2")
#conf.set('spark.sql.shuffle.partitions',400) 
#sc = SparkContext(conf=conf)

In [5]:
sqlct = SQLContext(sc)

In [6]:
loc='/data/MSA_8050_Spring_18/proj10'

In [7]:
%%sh
hdfs dfs -ls /data/MSA_8050_Spring_18/proj10/

Found 13 items
-rw-r--r--   3 pkalavai2 hdfs       4656 2018-04-28 17:19 /data/MSA_8050_Spring_18/proj10/sample-yelp-review.csv
-rw-r--r--   3 pkalavai2 hdfs   70015437 2018-04-29 20:13 /data/MSA_8050_Spring_18/proj10/sample100k.csv
-rw-r--r--   3 pkalavai2 hdfs    1873475 2018-04-28 18:41 /data/MSA_8050_Spring_18/proj10/sample2k.csv
-rw-r--r--   3 pkalavai2 hdfs    1873475 2018-04-29 21:37 /data/MSA_8050_Spring_18/proj10/sample2kn.csv
-rw-r--r--   3 pkalavai2 hdfs   31760674 2018-04-27 17:34 /data/MSA_8050_Spring_18/proj10/yelp_business.csv
-rw-r--r--   3 pkalavai2 hdfs   41377121 2018-04-27 17:34 /data/MSA_8050_Spring_18/proj10/yelp_business_attributes.csv
-rw-r--r--   3 pkalavai2 hdfs   13866351 2018-04-27 17:34 /data/MSA_8050_Spring_18/proj10/yelp_business_hours.csv
-rw-r--r--   3 pkalavai2 hdfs  135964892 2018-04-27 17:34 /data/MSA_8050_Spring_18/proj10/yelp_checkin.csv
-rw-r--r--   3 pkalavai2 hdfs 3791120545 2018-04-27 17:31 /data/MSA_8050_Spring_18/proj10/yelp_review.csv
-rw-r-

In [8]:
os.path.join(loc, 'yelp_review100k.csv')

'/data/MSA_8050_Spring_18/proj10/yelp_review100k.csv'

In [26]:
#%%sh
#hdfs dfs -get /data/MSA_8050_Spring_18/proj10/sample-yelp-review.csv

get: `sample-yelp-review.csv': File exists


In [33]:
#%%sh
#hdfs dfs -get /data/MSA_8050_Spring_18/proj10/yelp_business.csv

In [34]:
#pdf_business = pd.read_csv('yelp_business.csv')
#pdf_business = pdf_business.replace({'\n':'', '\t':'', '\r':''}, regex=True)

In [38]:
#%%sh
#hdfs dfs -get /data/MSA_8050_Spring_18/proj10/yelp_user.csv

In [39]:
#pdf_user = pd.read_csv('yelp_user.csv')
#pdf_user = pdf_user.replace({'\n':'', '\t':'', '\r':''}, regex=True)

In [121]:
#pdf_business.columns

# Reading yelp_review1 file. This file is the cleaned file (without \n characters)

In [9]:
df_rev = sqlct.read.format("com.databricks.spark.csv").options(charset="ISO-8859-1",separator = ',', multiline=True, header=True, inferSchema = True).load(os.path.join(loc, 'yelp_review1.csv'))

#### count of number of reviews - 5.2 million

In [10]:
#count of number of reviews - 5.2 million
df_rev.count()

5261668

In [14]:
df_rev.show(5)

+--------------------+--------------------+--------------------+-----+----------+--------------------+------+-----+----+
|           review_id|             user_id|         business_id|stars|      date|                text|useful|funny|cool|
+--------------------+--------------------+--------------------+-----+----------+--------------------+------+-----+----+
|vkVSCC7xljjrAI4UG...|bv2nCi5Qv5vroFiqK...|AEx2SYEUJmTxVVB18...|    5|2016-05-28|Super simple plac...|     0|    0|   0|
|n6QzIUObkYshz4dz2...|bv2nCi5Qv5vroFiqK...|VR6GpWIda3SfvPC-l...|    5|2016-05-28|Small unassuming ...|     0|    0|   0|
|MV3CcKScW05u5LVfF...|bv2nCi5Qv5vroFiqK...|CKC0-MOWMqoeWf6s-...|    5|2016-05-28|Lester's is locat...|     0|    0|   0|
|IXvOzsEMYtiJI0CAR...|bv2nCi5Qv5vroFiqK...|ACFtxLv8pGrrxMm6E...|    4|2016-05-28|Love coming here....|     0|    0|   0|
|L_9BTb55X0GDtThi6...|bv2nCi5Qv5vroFiqK...|s2I_Ni76bjJNK9yG6...|    4|2016-05-28|Had their chocola...|     0|    0|   0|
+--------------------+----------

## count of reviews based on the star rating

In [11]:
#count of reviews based on the star rating
df_rev.groupby("stars").count().show()

+-----+-------+
|stars|  count|
+-----+-------+
|    1| 731363|
|    2| 438161|
|    3| 615481|
|    4|1223316|
|    5|2253347|
+-----+-------+



In [12]:
####Number of reviews by user in descending order of count
pdf = df_rev.groupby("user_id").count().orderBy("count",ascending = False).show()

+--------------------+-----+
|             user_id|count|
+--------------------+-----+
|CxDOIDnH8gp9KXzpB...| 3569|
|bLbSNkLggFnqwNNzz...| 2077|
|PKEzKWv_FktMm2mGP...| 1611|
|DK57YibC5ShBmqQl9...| 1463|
|QJI9OSEn6ujRCtrX0...| 1322|
|d_TBs6J3twMy9GChq...| 1184|
|ELcQDlf69kb-ihJfx...| 1159|
|cMEtAiW60I5wE_vLf...| 1126|
|hWDybu_KvYLSdEFzG...| 1117|
|U4INQZOPSUaj8hMjL...| 1101|
|UYcmGbelzRa0Q6Jqz...| 1092|
|62GNFh5FySkA3MbrQ...| 1051|
|dIIKEfOgo0KqUfGQv...|  989|
|n86B7IkbU20AkxlFX...|  955|
|iDlkZO2iILS8Jwfdy...|  947|
|N3oNEwh0qgPqPP3Em...|  926|
|rCWrxuRC8_pfagpch...|  915|
|Ry1O_KXZHGRI8g5zB...|  896|
|0BBUmH7Krcax1RZgb...|  874|
|pMefTWo6gMdx8WhYS...|  838|
+--------------------+-----+
only showing top 20 rows



In [153]:
#This creates a csv o/p file: Do not run if cluster is already out of space.
#pdf1 = df_rev.groupby("user_id").count().orderBy("count",ascending = False).take(50).toPandas()
#pdf1.to_csv("Top50ReviewCountByUser.csv",index=False)

In [13]:
####Number of reviews each business_id in descending order of count
df_rev.groupby("business_id").count().orderBy("count",ascending = False).show()

+--------------------+-----+
|         business_id|count|
+--------------------+-----+
|4JNXUYY8wbaaDmk3B...| 7362|
|RESDUcs7fIiihp38-...| 7006|
|K7lWdNUhCbcnEvI0N...| 5951|
|cYwJA2A6I12KNkm2r...| 5448|
|DkYS3arLOhA8si5uU...| 4870|
|f4x1YBxkLrZg652xt...| 4774|
|2weQS-RnoOBhb1KsH...| 4018|
|KskYqH1Bi7Z_61pH6...| 3965|
|eoHdUeQDNgQ6WYEnP...| 3911|
|ujHiaprwCQ5ewziu0...| 3836|
|SMPbvZLSMMb7KU76Y...| 3794|
|5LNZ67Yw9RD6nf4_U...| 3772|
|iCQpiavjjPzJ5_3gP...| 3741|
|AV6weBrZFFBfRGCbc...| 3623|
|rcaPajgKOJC2vo_l3...| 3571|
|El4FC8jcawUVgw_0E...| 3445|
|FaHADZARwnY4yvlvp...| 3287|
|hihud--QRriCYZw1z...| 3264|
|Wxxvi3LZbHNIDwJ-Z...| 3103|
|7sPNbCx7vGAaH7SbN...| 3064|
+--------------------+-----+
only showing top 20 rows



In [155]:
#Creates a csv file with output. 
#pdf1 = df_rev.groupby("business_id").count().orderBy("count",ascending = False).take(50).toPandas()
#pdf1.to_csv("Top50ReviewCountByBusiness.csv",index=False)

## Sorted by the number of reviews of each ratings given by user.

In [14]:
#Sorted by the number of reviews of each ratings given by user. Business id is not included. 
df_rev.groupby("user_id","stars").count().orderBy(["count","user_id","stars"],ascending = [0,1,0]).show()

+--------------------+-----+-----+
|             user_id|stars|count|
+--------------------+-----+-----+
|CxDOIDnH8gp9KXzpB...|    3| 1924|
|CxDOIDnH8gp9KXzpB...|    4| 1120|
|bLbSNkLggFnqwNNzz...|    3|  979|
|DK57YibC5ShBmqQl9...|    4|  814|
|bLbSNkLggFnqwNNzz...|    4|  771|
|cMEtAiW60I5wE_vLf...|    4|  715|
|PKEzKWv_FktMm2mGP...|    4|  707|
|QJI9OSEn6ujRCtrX0...|    4|  630|
|PKEzKWv_FktMm2mGP...|    3|  601|
|hWDybu_KvYLSdEFzG...|    4|  527|
|62GNFh5FySkA3MbrQ...|    3|  524|
|d_TBs6J3twMy9GChq...|    4|  515|
|dIIKEfOgo0KqUfGQv...|    4|  481|
|n86B7IkbU20AkxlFX...|    4|  477|
|Wu0yySWcHQ5tZ_59H...|    3|  445|
|UYcmGbelzRa0Q6Jqz...|    4|  442|
|U4INQZOPSUaj8hMjL...|    4|  436|
|N3oNEwh0qgPqPP3Em...|    4|  432|
|Ry1O_KXZHGRI8g5zB...|    4|  430|
|d_TBs6J3twMy9GChq...|    3|  427|
+--------------------+-----+-----+
only showing top 20 rows



In [178]:
#pdf2 = df_rev.groupby("user_id","stars").count().orderBy(["count","user_id","stars"],ascending = [0,1,0]).take(50).toPandas()
#pdf2.to_csv("TopReviewCountByUserWithStarsDetails",index=False)

# Way to convert a DF to rdd

In [23]:
#Way to convert a DF to rdd
rdd = df_rev.rdd

#### mapping rating and review - sample code

In [26]:
#mapping rating and review
pairs = rdd.map(lambda x: (x[3],x[5]))
pairs.take(1)

[(5,
  u"Super simple place but amazing nonetheless. It's been around since the 30's and they still serve the same thing they started with: a bologna and salami sandwich with mustard. Staff was very helpful and friendly.")]

### create a list of punctuation characters to be replaced by space

In [162]:
#create a list of punctuation characters to be replaced by space
table = dict((ord(char), None) for char in string.punctuation)

In [161]:
#Create a list of stopwords. This list is not exhaustive. Needs to be updated whan a new stop word is identified.
stop1 = ["said", 'im',"went", "de",'got',"la","un","et","ive", 'go', 'my',' ','an','from','there','what','they','when','we','which','their','how', 'its', 'would', 'if', 'were', 'has', 'because', 'by', 'them', 'we', 'had', 'they', 'us', 'your', 'or', 'me', 'these', 'those', 'after', 'before', 'will', 'off', 'up', 'down', 'out', 'be', 'all', 'at', 'just', 'the', 'are', 'one', 'not', '', 'the', 'i', 'and', 'to', 'it', 'a', 'for', 'any', 'this', 'is', 'my', 'of', 'in', 'with', 'have', 'as', 'on', 'that', 'so', 'you', 'this', 'but', 'was', 'it']

from nltk.corpus import stopwords
stop2 = stopwords.words("english")
stop = stop1+stop2

In [179]:
#Python 3 code to remove punctuations
#translator = string.maketrans(string.punctuation, ' '*len(string.punctuation))
#text = "He is amazing.!isn't he?"
#print text.translate(translator)

In [224]:
def analyse_eachline(eachline):
    #Splitting the review after lowering and removing punctuations
    text = [w for w in eachline[5].lower().translate(table).split(" ") if w not in stop]
    return((word,1) for word in text)
top_words_rdd = rdd.flatMap(analyse_eachline).reduceByKey(lambda x, y: x+y)
top_words_rdd.take(100)

[(u'unmadei', 1),
 (u'usuallyand', 1),
 (u'allowedhoover', 1),
 (u'roughwent', 1),
 (u'nunnery', 6),
 (u'realsuper', 1),
 (u'ownershiphave', 1),
 (u'nallez', 27),
 (u'seeservice', 5),
 (u'diabetesheart', 1),
 (u'havefunlaughssnacksconversation', 1),
 (u'tfaya', 4),
 (u'cmonbutconfession', 1),
 (u'soupsalad18', 1),
 (u'standardfinal', 1),
 (u'adoptionvery', 1),
 (u'preferablebeing', 1),
 (u'rojovery', 1),
 (u'safewaysomething', 2),
 (u'cfb', 10),
 (u'thatsuite', 1),
 (u'notconcessions', 1),
 (u'mealperiod', 1),
 (u'paringyouve', 1),
 (u'piratesdont', 1),
 (u'cycologists', 2),
 (u'offendeddo', 1),
 (u'letmetakeoutasecondmortgage', 1),
 (u'considereda', 2),
 (u'ontariotoo', 1),
 (u'didntapologise', 1),
 (u'effortvery', 1),
 (u'gueulecaf\xe3\xa9', 1),
 (u'stapelt', 2),
 (u'anywaysfooddrink', 1),
 (u'towingrepo', 1),
 (u'screaming', 10999),
 (u'weell', 1),
 (u'mousquetairesqc', 1),
 (u'curvefrom', 1),
 (u'mehhhreally', 1),
 (u'placewatch', 1),
 (u'warmhei\xe3\x9f', 1),
 (u'multiorgan', 1),


In [225]:
final_list = top_words_rdd.sortBy(lambda x:x[1],ascending=False)

# Top 100 words from "all reviews"

In [227]:
#Top 100 words from "all reviews"
final_list.take(100)

[(u'place', 2581149),
 (u'food', 2576028),
 (u'good', 2545001),
 (u'great', 2248598),
 (u'like', 1885330),
 (u'service', 1810121),
 (u'time', 1772471),
 (u'get', 1760261),
 (u'back', 1534258),
 (u'really', 1368903),
 (u'also', 1135165),
 (u'dont', 1016651),
 (u'even', 1006224),
 (u'nice', 974749),
 (u'well', 914616),
 (u'best', 908324),
 (u'didnt', 844421),
 (u'always', 841008),
 (u'came', 829779),
 (u'love', 823516),
 (u'ordered', 816249),
 (u'staff', 809908),
 (u'first', 788669),
 (u'order', 768717),
 (u'little', 763957),
 (u'never', 762256),
 (u'friendly', 753934),
 (u'restaurant', 751087),
 (u'could', 739546),
 (u'much', 734349),
 (u'try', 733398),
 (u'people', 732080),
 (u'come', 731388),
 (u'definitely', 714829),
 (u'chicken', 707936),
 (u'make', 656569),
 (u'made', 650537),
 (u'experience', 649362),
 (u'pretty', 642130),
 (u'amazing', 640383),
 (u'menu', 631585),
 (u'going', 627682),
 (u'know', 620747),
 (u'delicious', 610674),
 (u'better', 605637),
 (u'way', 594908),
 (u'two', 

In [163]:
pairs = rdd.map(lambda x: (x[3],x[5].lower().translate(table)))
pairs1 = pairs.flatMapValues(lambda x: x.split(" ")).filter(lambda x: not x[1] in stop)
pairs2 = pairs1.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x+y)

In [117]:
pairs2.take(50)

[((4, u'dairyfreezer'), 1),
 ((3, u'batterthere'), 1),
 ((1, u'backworkers'), 3),
 ((4, u'drugsbut'), 1),
 ((5, u'bestmore'), 2),
 ((1, u'cashierassistant'), 1),
 ((4, u'selectionbetween'), 1),
 ((4, u'inrepair'), 1),
 ((5, u'tortellinigrand'), 1),
 ((4, u'vehiclesalso'), 1),
 ((1, u'downfuckinghill'), 1),
 ((3, u'biegnets'), 1),
 ((1, u'r\xe3\xa8gle'), 5),
 ((3, u'morsette'), 1),
 ((3, u'leaky'), 85),
 ((3, u'onionsone'), 2),
 ((5, u'omelettechicken'), 1),
 ((3, u'argueably'), 1),
 ((4, u'childof'), 1),
 ((4, u'sorely'), 308),
 ((2, u'yeey'), 2),
 ((5, u'sameol'), 5),
 ((2, u'awfulcommercial'), 1),
 ((4, u'quitany'), 1),
 ((5, u'probierenswert'), 1),
 ((2, u'bikeif'), 1),
 ((5, u'dow'), 37),
 ((1, u'l4l5'), 3),
 ((5, u'cheesecakecookies'), 1),
 ((5, u'restaurantreview'), 2),
 ((5, u'brewedgreat'), 1),
 ((1, u'palettefyi'), 1),
 ((4, u'overchargedmaybe'), 1),
 ((4, u'looktwo'), 2),
 ((3, u'supertasting'), 1),
 ((3, u'tidynow'), 1),
 ((4, u'envelope3'), 1),
 ((1, u'refil'), 30),
 ((5, u

#### Top words split by rating

In [165]:
pairs3 = pairs2.sortBy(lambda x:x[1],ascending=False)
pairs3.take(50)

[((5, u'great'), 1224604),
 ((5, u'place'), 1008902),
 ((4, u'good'), 867820),
 ((5, u'food'), 846899),
 ((5, u'good'), 765666),
 ((5, u'service'), 684897),
 ((5, u'time'), 662588),
 ((4, u'place'), 661854),
 ((4, u'food'), 656216),
 ((4, u'great'), 638370),
 ((5, u'best'), 560469),
 ((5, u'get'), 558592),
 ((5, u'like'), 546309),
 ((5, u'back'), 532997),
 ((5, u'love'), 487222),
 ((4, u'like'), 485185),
 ((3, u'good'), 479307),
 ((5, u'amazing'), 461273),
 ((5, u'really'), 458826),
 ((5, u'always'), 450665),
 ((5, u'also'), 450450),
 ((5, u'friendly'), 411745),
 ((3, u'food'), 406891),
 ((4, u'get'), 405082),
 ((4, u'really'), 404567),
 ((5, u'staff'), 389037),
 ((4, u'time'), 383977),
 ((4, u'service'), 373095),
 ((5, u'well'), 365983),
 ((1, u'get'), 362215),
 ((5, u'definitely'), 358904),
 ((1, u'food'), 349708),
 ((5, u'recommend'), 348527),
 ((3, u'place'), 342804),
 ((5, u'nice'), 342597),
 ((1, u'back'), 341183),
 ((1, u'service'), 337340),
 ((1, u'place'), 334570),
 ((5, u'del

# top50 words in 5* reviews

In [167]:
#top50 words in 5* reviews
pairs4 = pairs3.filter(lambda x: x[0][0] == 5)
pairs4.take(50)

[((5, u'great'), 1224604),
 ((5, u'place'), 1008902),
 ((5, u'food'), 846899),
 ((5, u'good'), 765666),
 ((5, u'service'), 684897),
 ((5, u'time'), 662588),
 ((5, u'best'), 560469),
 ((5, u'get'), 558592),
 ((5, u'like'), 546309),
 ((5, u'back'), 532997),
 ((5, u'love'), 487222),
 ((5, u'amazing'), 461273),
 ((5, u'really'), 458826),
 ((5, u'always'), 450665),
 ((5, u'also'), 450450),
 ((5, u'friendly'), 411745),
 ((5, u'staff'), 389037),
 ((5, u'well'), 365983),
 ((5, u'definitely'), 358904),
 ((5, u'recommend'), 348527),
 ((5, u'nice'), 342597),
 ((5, u'delicious'), 333940),
 ((5, u'even'), 331422),
 ((5, u'first'), 290362),
 ((5, u'try'), 285176),
 ((5, u'made'), 275549),
 ((5, u'experience'), 273807),
 ((5, u'dont'), 268513),
 ((5, u'new'), 263716),
 ((5, u'come'), 257232),
 ((5, u'everything'), 256873),
 ((5, u'make'), 256651),
 ((5, u'awesome'), 247757),
 ((5, u'came'), 244475),
 ((5, u'restaurant'), 243102),
 ((5, u'every'), 239920),
 ((5, u'ever'), 238966),
 ((5, u'never'), 236

# top50 words in 4* reviews

In [168]:
#top50 words in 4* reviews
pairs5 = pairs3.filter(lambda x: x[0][0] == 4)
pairs5.take(50)

[((4, u'good'), 867820),
 ((4, u'place'), 661854),
 ((4, u'food'), 656216),
 ((4, u'great'), 638370),
 ((4, u'like'), 485185),
 ((4, u'get'), 405082),
 ((4, u'really'), 404567),
 ((4, u'time'), 383977),
 ((4, u'service'), 373095),
 ((4, u'also'), 327395),
 ((4, u'back'), 322908),
 ((4, u'nice'), 320509),
 ((4, u'little'), 267705),
 ((4, u'well'), 252583),
 ((4, u'pretty'), 229618),
 ((4, u'dont'), 220597),
 ((4, u'definitely'), 220238),
 ((4, u'always'), 218707),
 ((4, u'chicken'), 218265),
 ((4, u'try'), 211095),
 ((4, u'ordered'), 209627),
 ((4, u'love'), 208236),
 ((4, u'delicious'), 204624),
 ((4, u'friendly'), 204157),
 ((4, u'menu'), 202521),
 ((4, u'restaurant'), 199042),
 ((4, u'much'), 181760),
 ((4, u'came'), 181643),
 ((4, u'even'), 181387),
 ((4, u'best'), 180879),
 ((4, u'order'), 179605),
 ((4, u'staff'), 176272),
 ((4, u'didnt'), 174570),
 ((4, u'come'), 172169),
 ((4, u'first'), 170673),
 ((4, u'bit'), 168366),
 ((4, u'people'), 164719),
 ((4, u'bar'), 158676),
 ((4, u'

# top50 words in 3* reviews

In [169]:
#top50 words in 3* reviews
pairs6 = pairs3.filter(lambda x: x[0][0] == 3)
pairs6.take(50)

[((3, u'good'), 479307),
 ((3, u'food'), 406891),
 ((3, u'place'), 342804),
 ((3, u'like'), 314063),
 ((3, u'get'), 239916),
 ((3, u'service'), 225705),
 ((3, u'really'), 224521),
 ((3, u'time'), 214104),
 ((3, u'great'), 202708),
 ((3, u'back'), 174496),
 ((3, u'nice'), 163167),
 ((3, u'pretty'), 156709),
 ((3, u'dont'), 153645),
 ((3, u'also'), 150111),
 ((3, u'ordered'), 144444),
 ((3, u'didnt'), 143416),
 ((3, u'little'), 139464),
 ((3, u'much'), 128070),
 ((3, u'came'), 123360),
 ((3, u'chicken'), 119346),
 ((3, u'order'), 117193),
 ((3, u'restaurant'), 117158),
 ((3, u'better'), 112940),
 ((3, u'well'), 112597),
 ((3, u'even'), 107412),
 ((3, u'menu'), 106883),
 ((3, u'try'), 106745),
 ((3, u'bit'), 105116),
 ((3, u'people'), 103799),
 ((3, u'wasnt'), 103584),
 ((3, u'come'), 101856),
 ((3, u'think'), 101383),
 ((3, u'could'), 100565),
 ((3, u'first'), 97917),
 ((3, u'though'), 89327),
 ((3, u'bar'), 87753),
 ((3, u'way'), 85479),
 ((3, u'bad'), 84087),
 ((3, u'still'), 83716),
 

# top50 words in 2* reviews

In [170]:
#top50 words in 2* reviews
pairs7 = pairs3.filter(lambda x: x[0][0] == 2)
pairs7.take(50)

[((2, u'food'), 316314),
 ((2, u'good'), 242054),
 ((2, u'like'), 234566),
 ((2, u'place'), 233019),
 ((2, u'get'), 194456),
 ((2, u'service'), 189084),
 ((2, u'time'), 181452),
 ((2, u'back'), 162674),
 ((2, u'really'), 146469),
 ((2, u'didnt'), 129797),
 ((2, u'dont'), 129459),
 ((2, u'ordered'), 125034),
 ((2, u'even'), 120361),
 ((2, u'came'), 109762),
 ((2, u'order'), 107202),
 ((2, u'great'), 101918),
 ((2, u'better'), 98351),
 ((2, u'much'), 96402),
 ((2, u'also'), 90933),
 ((2, u'restaurant'), 87707),
 ((2, u'could'), 87633),
 ((2, u'first'), 85901),
 ((2, u'people'), 85189),
 ((2, u'nice'), 82738),
 ((2, u'two'), 82738),
 ((2, u'minutes'), 82476),
 ((2, u'wasnt'), 79789),
 ((2, u'2'), 79679),
 ((2, u'chicken'), 79495),
 ((2, u'asked'), 77499),
 ((2, u'never'), 76213),
 ((2, u'well'), 75287),
 ((2, u'pretty'), 74813),
 ((2, u'way'), 73922),
 ((2, u'going'), 73053),
 ((2, u'bad'), 72680),
 ((2, u'come'), 72465),
 ((2, u'know'), 72071),
 ((2, u'table'), 68223),
 ((2, u'think'), 6

# top50 words in 1* reviews

In [171]:
#top20 words in 1* reviews
pairs8 = pairs3.filter(lambda x: x[0][0] == 1)
pairs8.take(50)

[((1, u'get'), 362215),
 ((1, u'food'), 349708),
 ((1, u'back'), 341183),
 ((1, u'service'), 337340),
 ((1, u'place'), 334570),
 ((1, u'time'), 330350),
 ((1, u'like'), 305207),
 ((1, u'never'), 269983),
 ((1, u'even'), 265642),
 ((1, u'told'), 255913),
 ((1, u'dont'), 244437),
 ((1, u'didnt'), 220487),
 ((1, u'asked'), 191198),
 ((1, u'could'), 191129),
 ((1, u'good'), 190154),
 ((1, u'order'), 182396),
 ((1, u'minutes'), 172731),
 ((1, u'came'), 170539),
 ((1, u'people'), 160692),
 ((1, u'customer'), 149563),
 ((1, u'going'), 144402),
 ((1, u'know'), 144212),
 ((1, u'first'), 143816),
 ((1, u'called'), 142970),
 ((1, u'ordered'), 141782),
 ((1, u'another'), 138221),
 ((1, u'ever'), 136457),
 ((1, u'really'), 134520),
 ((1, u'took'), 128685),
 ((1, u'bad'), 128424),
 ((1, u'manager'), 127705),
 ((1, u'come'), 127666),
 ((1, u'take'), 126429),
 ((1, u'car'), 125857),
 ((1, u'give'), 123892),
 ((1, u'way'), 120426),
 ((1, u'still'), 119532),
 ((1, u'experience'), 119430),
 ((1, u'2'), 1

# Top 50 words from reviews which were found useful by users.

In [240]:
upairs = rdd.map(lambda x: (x[6],x[5].lower().translate(table))).filter(lambda x: x[0] != 0)
upairs1 = upairs.flatMapValues(lambda x: x.split(" ")).filter(lambda x: not x[1] in stop)
upairs2 = upairs1.map(lambda x: (x[1], 1)).reduceByKey(lambda x, y: x+y)

In [241]:
upairs2.sortBy(lambda x:x[1],ascending=False).take(50)

[(u'place', 1417723),
 (u'good', 1340872),
 (u'food', 1298282),
 (u'like', 1169151),
 (u'get', 1072381),
 (u'great', 1070939),
 (u'time', 1016232),
 (u'service', 907100),
 (u'back', 873823),
 (u'really', 805784),
 (u'also', 678480),
 (u'dont', 635981),
 (u'even', 620299),
 (u'well', 536344),
 (u'nice', 534193),
 (u'didnt', 523360),
 (u'came', 487910),
 (u'first', 471772),
 (u'best', 462061),
 (u'could', 461172),
 (u'never', 457047),
 (u'ordered', 456697),
 (u'little', 455495),
 (u'people', 454733),
 (u'much', 448882),
 (u'order', 438614),
 (u'always', 429483),
 (u'love', 424308),
 (u'come', 421488),
 (u'restaurant', 421212),
 (u'try', 417133),
 (u'staff', 415288),
 (u'know', 400053),
 (u'make', 397293),
 (u'chicken', 387623),
 (u'pretty', 385789),
 (u'made', 383502),
 (u'going', 379358),
 (u'definitely', 372030),
 (u'friendly', 370925),
 (u'two', 369599),
 (u'experience', 368363),
 (u'menu', 367908),
 (u'way', 367190),
 (u'day', 352815),
 (u'new', 349687),
 (u'better', 348926),
 (u'wan

In [246]:
useful_words = upairs1.map(lambda x: x[1]).collect()

[u'cycle',
 u'pub',
 u'las',
 u'vegas',
 u'blast',
 u'groupon',
 u'rented',
 u'bike',
 u'11',
 u'afternoon']

# Top 50 words from reviews which were found funny by users.

In [242]:
fpairs = rdd.map(lambda x: (x[7],x[5].lower().translate(table))).filter(lambda x: x[0] != 0)
fpairs1 = fpairs.flatMapValues(lambda x: x.split(" ")).filter(lambda x: not x[1] in stop)
fpairs2 = fpairs1.map(lambda x: (x[1], 1)).reduceByKey(lambda x, y: x+y)

In [243]:
fpairs2.sortBy(lambda x:x[1],ascending=False).take(50)

[(u'place', 696780),
 (u'like', 649445),
 (u'good', 649442),
 (u'food', 632718),
 (u'get', 557701),
 (u'time', 489718),
 (u'great', 446013),
 (u'back', 419716),
 (u'really', 405103),
 (u'service', 399552),
 (u'dont', 351051),
 (u'also', 326414),
 (u'even', 320754),
 (u'didnt', 274029),
 (u'well', 262445),
 (u'nice', 249395),
 (u'people', 246081),
 (u'came', 241960),
 (u'could', 235458),
 (u'little', 232871),
 (u'first', 232654),
 (u'much', 232330),
 (u'ordered', 228472),
 (u'order', 227915),
 (u'never', 227147),
 (u'know', 221801),
 (u'come', 212825),
 (u'best', 207762),
 (u'restaurant', 207595),
 (u'pretty', 205676),
 (u'make', 202132),
 (u'love', 201388),
 (u'try', 201333),
 (u'chicken', 196616),
 (u'way', 194778),
 (u'always', 193428),
 (u'going', 191280),
 (u'two', 190707),
 (u'made', 188982),
 (u'want', 185314),
 (u'menu', 184224),
 (u'think', 179438),
 (u'see', 177538),
 (u'better', 177178),
 (u'right', 174865),
 (u'staff', 174840),
 (u'day', 171713),
 (u'around', 171711),
 (u'st

In [244]:
cpairs = rdd.map(lambda x: (x[8],x[5].lower().translate(table))).filter(lambda x: x[0] != 0)
cpairs1 = cpairs.flatMapValues(lambda x: x.split(" ")).filter(lambda x: not x[1] in stop)
cpairs2 = cpairs1.map(lambda x: (x[1], 1)).reduceByKey(lambda x, y: x+y)
cpairs2.sortBy(lambda x:x[1],ascending=False).take(50)

[(u'place', 838978),
 (u'good', 805174),
 (u'food', 738641),
 (u'like', 701035),
 (u'great', 644093),
 (u'get', 596446),
 (u'time', 550122),
 (u'really', 487788),
 (u'back', 461141),
 (u'service', 457898),
 (u'also', 413008),
 (u'dont', 351520),
 (u'even', 332355),
 (u'nice', 320564),
 (u'well', 319791),
 (u'little', 289126),
 (u'best', 281819),
 (u'love', 279218),
 (u'didnt', 271672),
 (u'much', 264495),
 (u'came', 264449),
 (u'first', 261084),
 (u'always', 259319),
 (u'ordered', 258261),
 (u'try', 257874),
 (u'people', 255741),
 (u'restaurant', 246073),
 (u'pretty', 243988),
 (u'come', 241806),
 (u'could', 240987),
 (u'order', 240649),
 (u'chicken', 238236),
 (u'menu', 235801),
 (u'definitely', 233416),
 (u'know', 226280),
 (u'make', 225957),
 (u'friendly', 222007),
 (u'made', 221833),
 (u'staff', 219481),
 (u'never', 218988),
 (u'way', 207848),
 (u'two', 205094),
 (u'delicious', 204276),
 (u'going', 200734),
 (u'right', 199901),
 (u'bar', 198100),
 (u'think', 196556),
 (u'see', 1957

# Average length of reviews by User_id.

In [172]:
####Average length of reviews by User_id.
ppairs = rdd.map(lambda x: (x[1], len(x[5].lower().translate(table).split(" "))))

#Create DataFrame
revLen_DF = ppairs.toDF(["User_id", "length"])
revLen_DF.groupBy("User_id").agg(avg("length")).orderBy("avg(length)",ascending=False).show()


+--------------------+-----------+
|             User_id|avg(length)|
+--------------------+-----------+
|quWDReh4dpERjCw2c...|     1149.0|
|vJYx1u5I0GCld02kk...|     1093.0|
|BG1XEbXfUGv8ejFro...|     1084.0|
|OXx7XlG1kOEw9T2_R...|     1077.0|
|fEou-_aC_U5bOEchX...|     1062.0|
|QR3qVDepUs7YpgsAX...|     1054.0|
|UyDpNcErsWlk8ATEX...|     1053.0|
|0x6xNj-v8MOpX_UoZ...|     1050.0|
|Si-WHeYLNb0MuvptM...|     1043.0|
|4vW84mfUC5ByVm6Ov...|     1043.0|
|ud4vRYUT8ZrMpmAmn...|     1042.0|
|cUEgOroKhyFBSxrEG...|     1041.0|
|kiQDaLW_hH9LzoHmL...|     1037.0|
|C5Ak5YjzztEvY8FiH...|     1032.0|
|cfZu5Aj43tvlnFYo5...|     1028.0|
|sgkGYlAiUCFKObgak...|     1026.0|
|SS2rd6nFaQuViS-CY...|     1025.0|
|xYz3TKbCR75tOgtyW...|     1023.0|
|ZHHCQQijPbWAxir2a...|     1023.0|
|5Y4bQEYp_3HXDvyOp...|     1022.0|
+--------------------+-----------+
only showing top 20 rows



In [180]:
#pdf1=revLen_DF.groupBy("User_id").agg(avg("length")).orderBy("avg(length)",ascending=False).take(50).toPandas()
#pdf1.to_csv("AvgLenOfRev-UserID",index=False)

# Average length of reviews for each business(top)

In [127]:
#Average length of reviews for each business
bpairs = rdd.map(lambda x: (x[2], len(x[5].lower().translate(table).split(" "))))

#Create DataFrame
revLen_DF = bpairs.toDF(["business_id", "length"])
revLen_DF.groupBy("business_id").agg(avg("length")).orderBy("avg(length)",ascending=False).show()

+--------------------+-----------------+
|         business_id|      avg(length)|
+--------------------+-----------------+
|80fy5ngVEX5_WJbK7...|            811.0|
|HZqISq3oXnibkQQ6_...|            771.0|
|WfITErw6wT8HrzQdM...|           672.75|
|_e764NHx1rGdh85UK...|672.3333333333334|
|hFNyMFD0_6TCuDEs9...|632.6666666666666|
|eNtb45z69XiH9P4ZJ...|            632.0|
|ar9xoFgFJK_f8gG23...|629.3333333333334|
|s0UrfuSYgukzch1th...|622.3333333333334|
|SrsUE1_g02QPaACc1...|618.6666666666666|
|0k0kwI4U8oUl-qEjR...|            614.0|
|VQNGyIwxXot4nPV0Y...|           610.75|
|9YZEuN2bsn8-_nduc...|            599.2|
|uZd7UVMib1EYUFFbf...|            599.0|
|2Q9VJZHgdkuLQeseX...|594.6666666666666|
|E-2cNh0sfIjRwRKvu...|            588.0|
|vy9dZu9jw0Y-JLtwN...|578.6666666666666|
|yI_QrxrKERNq2efLo...|           575.75|
|yO8XnZDmxnTtR03sS...|572.6666666666666|
|24ASBsYjwEWEeHjoc...|571.6666666666666|
|VPVRh3inT4ApMXGH0...|571.6666666666666|
+--------------------+-----------------+
only showing top

## Average review length for each star rating. Surprisingly 5 * ratings are shorter than 1 * ratings.

In [128]:
##Average review length for each star rating. Surprisingly 5 * ratings are shorter than 1 * ratings. 
spairs = rdd.map(lambda x: (x[3], len(x[5].lower().translate(table).split(" "))))

#Create DataFrame
revLen_DF = spairs.toDF(["stars", "length"])
revLen_DF.groupBy("stars").agg(avg("length")).orderBy("avg(length)",ascending=False).show()

+-----+------------------+
|stars|       avg(length)|
+-----+------------------+
|    1| 145.0898459451736|
|    2|144.46203336216595|
|    3|133.63201138621665|
|    4| 116.9679755680462|
|    5| 91.14450992235106|
+-----+------------------+



In [152]:
#pdf=revLen_DF.groupBy("stars").agg(avg("length")).orderBy("avg(length)",ascending=False).toPandas()
#pdf.head()

,stars,avg(length)
0,1,145.089846
1,2,144.462033
2,3,133.632011
3,4,116.967976
4,5,91.144510


In [176]:
df_rev.show(5)

+--------------------+--------------------+--------------------+-----+----------+--------------------+------+-----+----+
|           review_id|             user_id|         business_id|stars|      date|                text|useful|funny|cool|
+--------------------+--------------------+--------------------+-----+----------+--------------------+------+-----+----+
|vkVSCC7xljjrAI4UG...|bv2nCi5Qv5vroFiqK...|AEx2SYEUJmTxVVB18...|    5|2016-05-28|Super simple plac...|     0|    0|   0|
|n6QzIUObkYshz4dz2...|bv2nCi5Qv5vroFiqK...|VR6GpWIda3SfvPC-l...|    5|2016-05-28|Small unassuming ...|     0|    0|   0|
|MV3CcKScW05u5LVfF...|bv2nCi5Qv5vroFiqK...|CKC0-MOWMqoeWf6s-...|    5|2016-05-28|Lester's is locat...|     0|    0|   0|
|IXvOzsEMYtiJI0CAR...|bv2nCi5Qv5vroFiqK...|ACFtxLv8pGrrxMm6E...|    4|2016-05-28|Love coming here....|     0|    0|   0|
|L_9BTb55X0GDtThi6...|bv2nCi5Qv5vroFiqK...|s2I_Ni76bjJNK9yG6...|    4|2016-05-28|Had their chocola...|     0|    0|   0|
+--------------------+----------

In [ ]:
#from pyspark.sql.functions import col
#dataset = sqlContext.range(0, 100).select((col("id") % 3).alias("key"))
#sampled = dataset.sampleBy("key", fractions={0: 0.1, 1: 0.2}, seed=0)
#sampled.groupBy("key").count().orderBy("key").show()

In [187]:
from pyspark.sql.functions import date_format

# On what days of the week do users write reviews? 

In [193]:
df_rev.select("user_id",date_format('date','EEEE').alias('day')).groupby("user_id","day").count().orderBy("count",ascending=False).show()

+--------------------+---------+-----+
|             user_id|      day|count|
+--------------------+---------+-----+
|CxDOIDnH8gp9KXzpB...| Saturday|  719|
|CxDOIDnH8gp9KXzpB...|   Friday|  699|
|CxDOIDnH8gp9KXzpB...| Thursday|  567|
|CxDOIDnH8gp9KXzpB...|Wednesday|  462|
|CxDOIDnH8gp9KXzpB...|  Tuesday|  460|
|CxDOIDnH8gp9KXzpB...|   Monday|  432|
|bLbSNkLggFnqwNNzz...|   Sunday|  391|
|DK57YibC5ShBmqQl9...|   Sunday|  316|
|bLbSNkLggFnqwNNzz...| Saturday|  305|
|rCWrxuRC8_pfagpch...|   Sunday|  298|
|bLbSNkLggFnqwNNzz...|   Friday|  285|
|bLbSNkLggFnqwNNzz...|   Monday|  282|
|PKEzKWv_FktMm2mGP...|Wednesday|  281|
|bLbSNkLggFnqwNNzz...|Wednesday|  281|
|bLbSNkLggFnqwNNzz...| Thursday|  267|
|bLbSNkLggFnqwNNzz...|  Tuesday|  266|
|d_TBs6J3twMy9GChq...|   Sunday|  259|
|cMEtAiW60I5wE_vLf...|   Monday|  258|
|DK57YibC5ShBmqQl9...| Saturday|  258|
|U4INQZOPSUaj8hMjL...|   Monday|  253|
+--------------------+---------+-----+
only showing top 20 rows



# Distribution of number of reviews on each day of the week
#Contrary to the belief, more number of reviews are written on Sundays closely followed by Mondays and not saturdays.
#Probably it is a reactive process. People go to different places on weekends and leisurely write reviews on the following days.
#People are busy on Fridays

In [204]:
#Distribution of number of reviews on each day of the week
df_rev.select("date",date_format('date','EEEE').alias('day')).groupby("day").count().orderBy("count",ascending=False).show()

+---------+------+
|      day| count|
+---------+------+
|   Sunday|794745|
|   Monday|793559|
|Wednesday|753938|
| Saturday|749882|
|  Tuesday|747057|
| Thursday|712783|
|   Friday|709704|
+---------+------+



In [211]:
df_rev.groupby("date").count().orderBy("count",ascending=True).show()

+----------+-----+
|      date|count|
+----------+-----+
|2005-07-08|    1|
|2005-05-02|    1|
|2005-05-09|    1|
|2005-07-05|    1|
|2005-07-03|    1|
|2004-07-22|    1|
|2005-06-09|    1|
|2005-11-10|    1|
|2005-10-10|    1|
|2005-11-15|    1|
|2005-10-05|    1|
|2005-08-09|    1|
|2005-12-11|    1|
|2005-06-07|    1|
|2005-09-09|    1|
|2005-12-13|    1|
|2005-09-13|    1|
|2005-08-04|    1|
|2004-10-25|    1|
|2006-03-23|    1|
+----------+-----+
only showing top 20 rows



In [212]:
df_rev.groupby("date").count().orderBy("count",ascending=False).show()

+----------+-----+
|      date|count|
+----------+-----+
|2017-08-12| 4614|
|2017-08-13| 4340|
|2017-07-14| 4269|
|2017-07-16| 4185|
|2017-07-08| 4159|
|2017-07-30| 4038|
|2017-07-09| 4025|
|2017-07-15| 3998|
|2017-07-29| 3973|
|2017-10-09| 3949|
|2017-06-25| 3938|
|2017-08-06| 3922|
|2017-06-09| 3912|
|2017-10-08| 3893|
|2017-07-22| 3845|
|2017-07-23| 3838|
|2017-08-05| 3832|
|2017-02-19| 3831|
|2017-09-17| 3826|
|2017-11-07| 3816|
+----------+-----+
only showing top 20 rows



In [213]:
from pyspark.sql.functions import year, month, dayofmonth

# The number of users giving reviews is increasing every year. Increase in businesses, increase in technology and many more reasons 

In [217]:
df_rev.select(year("date").alias('year')).groupby("year").count().show()

+----+-------+
|year|  count|
+----+-------+
|2004|     14|
|2005|    870|
|2006|   5669|
|2007|  23020|
|2008|  61553|
|2009|  98288|
|2010| 187073|
|2011| 290933|
|2012| 350381|
|2013| 472595|
|2014| 678351|
|2015| 911487|
|2016|1052916|
|2017|1128518|
+----+-------+



In [218]:
df_rev.select(month("date").alias('month')).groupby("month").count().show()

+-----+------+
|month| count|
+-----+------+
|    1|410760|
|    2|382709|
|    3|433304|
|    4|428625|
|    5|442017|
|    6|454042|
|    7|506013|
|    8|503860|
|    9|449790|
|   10|448157|
|   11|428940|
|   12|373451|
+-----+------+



# Sentiment Analysis for a business which has almost 7000 reviews.

In [248]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

/usr/lib/python2.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [247]:
senti = rdd.filter(lambda x: x[2]=='4JNXUYY8wbaaDmk3BPzlWw').map(lambda x: (x[3],round(sid.polarity_scores(x[5])['compound'],1)))

In [250]:
sentiDF = senti.toDF(["stars", "senti_score"])

In [249]:
senti.take(3)

[(4, 1.0), (5, 1.0), (2, 0.0)]

In [251]:
sentiDF.groupby("stars").agg(avg("senti_score")).show()

+-----+--------------------+
|stars|    avg(senti_score)|
+-----+--------------------+
|    1|-0.00723404255319...|
|    2|   0.413429256594724|
|    3|  0.7460187353629977|
|    4|  0.8781832298136647|
|    5|  0.9139939024390246|
+-----+--------------------+



# Average sentiment of all the reviews in the review file:

In [252]:
total_senti = rdd.map(lambda x: (x[3],round(sid.polarity_scores(x[5])['compound'],1)))

In [253]:
totalSentiDF = total_senti.toDF(["stars", "senti_score"])

In [256]:
totalSentiDF.groupby("stars").agg(avg("senti_score")).show()

+-----+--------------------+
|stars|    avg(senti_score)|
+-----+--------------------+
|    1|-0.15963796363775978|
|    2| 0.30377715040818515|
|    3|  0.6406932139253102|
|    4|  0.8317341553615031|
|    5|  0.8698276829979905|
+-----+--------------------+



In [130]:
####business dataset
df_bus = sqlct.read.format("com.databricks.spark.csv").options(charset="ISO-8859-1",separator = ',', multiline=True, header=True, inferSchema = True).load(os.path.join(loc, 'yelp_business.csv'))

In [130]:
####business dataset
df_bus = sqlct.read.format("com.databricks.spark.csv").options(charset="ISO-8859-1",separator = ',', multiline=True, header=True, inferSchema = True).load(os.path.join(loc, 'yelp_business.csv'))

In [16]:
df_bus.count()

174567

In [131]:
df_user = sqlct.read.format("com.databricks.spark.csv").options(charset="ISO-8859-1",separator = ',', multiline=True, header=True, inferSchema = True).load(os.path.join(loc, 'yelp_user.csv'))

In [18]:
df_user.count()

1326100

In [133]:
df_bus.head()

Row(business_id=u'FYWN1wneV18bWNgQjJ2GNg', name=u'"Dental by Design"', neighborhood=u'', address=u'"4855 E Warner Rd, Ste B9"', city=u'Ahwatukee', state=u'AZ', postal_code=u'85044', latitude=33.3306902, longitude=-111.9785992, stars=4.0, review_count=22, is_open=1, categories=u'Dentists;General Dentistry;Health & Medical;Oral Surgeons;Cosmetic Dentists;Orthodontists')

In [139]:
df_user.head()

Row(user_id=u'JJ-aSuM4pCFPdkfoZ34q0Q', name=u'Chris', review_count=10, yelping_since=u'2013-09-24', friends=u'0njfJmB-7n84DlIgUByCNw, rFn3Xe3RqHxRSxWOU19Gpg, HVUAmApa0fCbHHVJ0ALshw, LBOTb6bJjCdFyDLNswUGmA, cy3d0moQOsrhWo6VAyA_kA, XTDeelhFzFX0h_8YELJDIQ, _erxdUfB8yJA_MCDBAaD_w, 3wMYVoHy15nxOCLZYYIhqA, 74C8Mdn3ISlFIwNZQUPEdg, Vc980a_kTQL9tzfG1ESq_g, UPld_8xdzvYmIMA4YxtG0A, ggppqaSt-4E-Y2XUVv6mMw, vILp0ctTM3fX0ucJeBw9Hg, Gl46kwumd4_f102dxgswDw, i9CDu8j-qcUfZYl67WBwiw, 23lcgZUt6dvtMqV_AZe9KQ, zacONW61-GjaCF9h5_icgA, BclKLmGPYlMEVr3OZ_Uuzw, ol00uKESTC5PR_fETT27AA, o7e8zecvSlPNTrPi1ZQLzg, 00JBHX0npVAC-itaFOUhpQ, im3w0wt-ZeE-lDa1zSdyMg, J_hT3mbtiwHIEWjTBSJnzQ, _54jmZK9vr9CN-SUSjFbMg, 65w6GtrNNlZDowVDAYGZFQ, WJeipqBVdQmVhnYrXlLb7A, vAhyONgVtU-oIaSHRJW0IA, Soa5S7dQsSjlG3lbTVGb2A, xtVcWcIKVORcBKhmdbrFiA, 7CghpxENlXnjTGOyNtRDsg, jI5mX0tNR79J_qoFKZMPdA, MFtiq9lbm95WVg501wNsww, SCICexHST-LMCowvH9h7jQ, SD2VfVLfSkz8m18dvo0HGA, Y8TLRIPm_ecnZPItgAJ3Kg, t9bfJI2zvx-3eFrN2zvQnw, 6yUCCifCroUPfXaVo5fCAA, SG

In [138]:
df_user.show(1)

+--------------------+-----+------------+-------------+--------------------+------+-----+----+----+-----+-------------+--------------+---------------+------------------+---------------+---------------+---------------+----------------+---------------+----------------+-----------------+-----------------+
|             user_id| name|review_count|yelping_since|             friends|useful|funny|cool|fans|elite|average_stars|compliment_hot|compliment_more|compliment_profile|compliment_cute|compliment_list|compliment_note|compliment_plain|compliment_cool|compliment_funny|compliment_writer|compliment_photos|
+--------------------+-----+------------+-------------+--------------------+------+-----+----+----+-----+-------------+--------------+---------------+------------------+---------------+---------------+---------------+----------------+---------------+----------------+-----------------+-----------------+
|JJ-aSuM4pCFPdkfoZ...|Chris|          10|   2013-09-24|0njfJmB-7n84DlIgU...|     0|    0

In [19]:
#df_user = sqlct.createDataFrame(pdf_user)
#df_rev = sqlct.createDataFrame(pdf_review)

In [140]:
bus = df_bus.alias('bus')
rev = df_rev.alias('rev')
df_rev_bus = rev.join(bus, rev.business_id == bus.business_id,how='left')

In [25]:
df_rev.orderBy(df_rev.date.desc()).show()

+--------------------+--------------------+--------------------+-----+----------+--------------------+------+-----+----+
|           review_id|             user_id|         business_id|stars|      date|                text|useful|funny|cool|
+--------------------+--------------------+--------------------+-----+----------+--------------------+------+-----+----+
|vR3s24M2fcsh8xHle...|85Jq6l707HWj2XpFw...|oLU_U-INLzjnZ4yW-...|    4|2017-12-11|wow les nombres d...|     1|    0|   0|
|2V1ZTOWtC6N4JadH6...|VBT4-K2XKqh1KW_vK...|HUwzarG0givckxXWT...|    5|2017-12-11|Great food, with ...|     0|    0|   0|
|zxfumrdFXha1NIZD2...|qy79NsGGJZq_nJkrE...|RRWJ1l8YDNYKh8LV7...|    4|2017-12-10|It's a good place...|     0|    0|   0|
|Qc6UAelXAJ9K_XRyo...|AvQvV4vylIwCiyS6s...|uJEot284h2qJbrWOg...|    5|2017-12-08|We really enjoyed...|     0|    0|   0|
|SrGj25VWMh-ZS5oI5...|CZs1Eq7t1fqpXV1yo...|eJPckkV1k7gn0V2aw...|    5|2017-12-07|Don't be sideline...|     1|    0|   0|
|5XMDyw2E4lM98w4By...|CZs1Eq7t1f